In [1]:
pip install scikit-learn nltk pandas numpy imbalanced-learn seaborn matplotlib

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

In [3]:
!pip install nltk
import nltk
nltk.download('punkt_tab') # Download the punkt_tab data package
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:

# Mount Google Drive
drive.mount('/content/drive')

# Path ke file di Google Drive
file_path = '/content/drive/My Drive/dicoding/ulasan.csv'

# Membaca file CSV
data = pd.read_csv(file_path)
print(data.info())
print(data.head())


Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6018 entries, 0 to 6017
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   at        6018 non-null   object
 1   userName  6018 non-null   object
 2   score     6018 non-null   int64 
 3   content   6018 non-null   object
dtypes: int64(1), object(3)
memory usage: 188.2+ KB
None
                    at                userName  score  \
0  2024-11-22 03:25:50         Badrin Tholchah      5   
1  2024-11-21 17:58:03  Cevin Agista Pramudito      5   
2  2024-11-21 15:30:17            Nur Kholikoh      1   
3  2024-11-21 01:00:19           Xavier Borkan      5   
4  2024-11-20 05:45:10           Andri Chandra      5   

                                             content  
0                             good taste and service  
1    Aplikasinya bagus dan sangat membantu pembelian  
2  Hati-hati aplikasi ini melakukan phising. Memi...  
3                   

In [5]:
# Download required NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
SLANG_DICT = {
    # Negative sentiments
    'gak': 'tidak', 'ga': 'tidak', 'nggak': 'tidak', 'ngga': 'tidak',
    'gk': 'tidak', 'g': 'tidak', 'kgk': 'tidak',
    'buruk': 'buruk', 'jelek': 'buruk', 'parah': 'buruk',
    'ancur': 'buruk', 'hancur': 'buruk', 'zonk': 'buruk',

    # Positive sentiments
    'mantap': 'bagus', 'mantul': 'bagus', 'manteb': 'bagus',
    'oke': 'baik', 'ok': 'baik', 'okee': 'baik', 'okey': 'baik',
    'sip': 'baik', 'asiap': 'baik', 'asik': 'baik',
    'enak': 'bagus', 'enk': 'bagus',
    'bgt': 'sangat', 'banget': 'sangat', 'bgets': 'sangat',
    'recommended': 'direkomendasikan', 'rekomen': 'direkomendasikan',
    'worth': 'sepadan', 'worthit': 'sepadan',

    # Service related
    'fast': 'cepat', 'quick': 'cepat', 'lambat': 'lama',
    'respon': 'respons', 'response': 'respons',
    'ramah': 'baik', 'friendly': 'baik',

    # Common expressions
    'thx': 'terima kasih', 'thanks': 'terima kasih', 'tks': 'terima kasih',
    'tq': 'terima kasih', 'ty': 'terima kasih'
}

In [7]:
def preprocess_text(text):
    """Enhanced text preprocessing"""
    if pd.isna(text):
        return ""

    # Convert to lowercase
    text = str(text).lower()

    # Remove URLs and HTML tags
    text = re.sub(r'http\S+|www\S+|https\S+|<.*?>', '', text)

    # Handle repeated characters (e.g., 'baguuuusss' -> 'bagus')
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Preserve important punctuation patterns
    text = re.sub(r'[!]{2,}', ' sangat ', text)  # Multiple exclamation marks indicate intensity
    text = re.sub(r'[?]{2,}', ' bingung ', text)  # Multiple question marks might indicate confusion

    # Handle emoticons
    text = re.sub(r'[:;][\-]?[)dp]', ' senang ', text)  # Happy emoticons
    text = re.sub(r'[:;][\-]?[\(]', ' sedih ', text)    # Sad emoticons

    # Tokenize
    words = word_tokenize(text)

    # Apply slang conversion
    words = [SLANG_DICT.get(word, word) for word in words]

    # Handle negation
    negation_words = {'tidak', 'bukan', 'kurang', 'belum'}
    processed_words = []
    negate = False

    for word in words:
        if word in negation_words:
            negate = True
            continue
        if negate:
            word = f"tidak_{word}"
            negate = False
        processed_words.append(word)

    # Remove stopwords but preserve important ones
    stop_words = set(stopwords.words('indonesian'))
    custom_stopwords = {'yg', 'dgn', 'ny', 'tk', 'klo', 'gk', 'krn', 'lg', 'sih', 'nya', 'deh', 'sih'}
    stop_words.update(custom_stopwords)
    important_words = {'tidak', 'sangat', 'kurang', 'bukan', 'belum', 'baik', 'buruk', 'bagus', 'cepat', 'lama'}

    words = [word for word in processed_words if (word not in stop_words or word in important_words)]

    return ' '.join(words)

In [8]:
# Enhanced sentiment labeling with confidence threshold
def label_sentiment(score):
    if score >= 4:
        return 'positive'
    elif score <= 2:
        return 'negative'
    else:
        return 'neutral'

In [9]:
from sklearn.ensemble import RandomForestClassifier
def train_sentiment_model(data):
    """Train sentiment model with improved handling of class imbalance"""
    print("Preprocessing text...")
    data['cleaned_content'] = data['content'].apply(preprocess_text)
    data['sentiment'] = data['score'].apply(label_sentiment)

    # Prepare data
    X = data['cleaned_content']
    y = data['sentiment']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Enhanced TF-IDF vectorization
    print("Vectorizing text...")
    vectorizer = TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 3),  # Include trigrams
        min_df=2,
        max_df=0.95,
        strip_accents='unicode',
        sublinear_tf=True
    )

    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)

    # Apply SMOTE for better balance
    print("Balancing dataset...")
    smote = SMOTE(random_state=42, k_neighbors=5)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_vectorized, y_train)

    # Use RandomForest with optimized parameters
    print("Training model...")
    model = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        class_weight='balanced',
        n_jobs=-1,
        random_state=42
    )

    model.fit(X_train_balanced, y_train_balanced)
    # Evaluate
    print("\nModel Evaluation:")
    y_pred = model.predict(X_test_vectorized)
    print(classification_report(y_test, y_pred))

    return model, vectorizer

In [10]:
# Main execution
def main():
    # Read data
    # Train model
    model, vectorizer = train_sentiment_model(data)

    # Print sample predictions
    print("\nSample Predictions:")
    sample_reviews = data['content'].head()
    for review in sample_reviews:
        cleaned = preprocess_text(review)
        vectorized = vectorizer.transform([cleaned])
        prediction = model.predict(vectorized)[0]
        print(f"\nReview: {review}")
        print(f"Sentiment: {prediction}")

if __name__ == "__main__":
    main()

Preprocessing text...
Vectorizing text...
Balancing dataset...
Training model...

Model Evaluation:
              precision    recall  f1-score   support

    negative       0.65      0.62      0.64       192
     neutral       0.10      0.10      0.10        41
    positive       0.92      0.93      0.93       971

    accuracy                           0.85      1204
   macro avg       0.56      0.55      0.55      1204
weighted avg       0.85      0.85      0.85      1204


Sample Predictions:

Review: good taste and service
Sentiment: positive

Review: Aplikasinya bagus dan sangat membantu pembelian
Sentiment: positive

Review: Hati-hati aplikasi ini melakukan phising. Meminta otp dan login ke akun telegram saya lalu menyebarkan pesan berisi link phising juga. Hati2 bila data anda disalahgunakan!!
Sentiment: negative

Review: ok terbaik
Sentiment: positive

Review: good
Sentiment: positive
